<a href="https://colab.research.google.com/github/rpolea/CE888/blob/main/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np

In [6]:
df= pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv',header=None)

In [7]:
df.head(10)
df.shape

(24983, 101)

In [8]:
df = df.replace(99, np.nan)

In [9]:
df = df.drop([0], axis = 1)

In [10]:
df.describe()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
count,16452.000000,18184.000000,16642.000000,15608.000000,24977.000000,20431.000000,24976.000000,24981.000000,15752.000000,20324.000000,21821.000000,22423.000000,24981.000000,22515.000000,24977.000000,24975.000000,24980.000000,24979.000000,24978.000000,24980.000000,24921.000000,21433.000000,20033.000000,15879.000000,20816.000000,23750.000000,24904.000000,23990.000000,24934.000000,17977.000000,24656.000000,24948.000000,16704.000000,21563.000000,24952.000000,24965.00000,16881.000000,22858.000000,23080.000000,22398.000000,...,24805.000000,24936.000000,20164.000000,17276.000000,24728.000000,24923.000000,17493.000000,24940.000000,24916.000000,20222.000000,8532.000000,8693.000000,8616.000000,8761.000000,8793.000000,8860.000000,8944.000000,9021.000000,9074.000000,9024.000000,9223.000000,9124.000000,9286.000000,9297.000000,9368.000000,9469.000000,9452.000000,9604.000000,9564.000000,9728.000000,9672.000000,9889.000000,9988.000000,10200.000000,10265.000000,10370.000000,10520.000000,10666.000000,10910.000000,9943.000000
mean,0.904573,0.208460,0.316574,-1.449114,0.403556,1.594447,-0.429662,-0.624085,-0.557031,1.301231,1.805797,1.449290,-1.762882,1.396334,-1.706201,-3.104818,-1.115141,-0.635103,0.154227,-0.933997,2.095626,0.868662,0.109192,-1.687420,0.382373,1.286232,3.189821,1.518830,2.972455,-0.436493,2.198943,3.155988,-1.368648,0.859494,3.006351,3.31250,-1.363987,1.300479,1.067337,0.998635,...,2.459180,2.976021,0.246998,-0.667938,2.273820,2.541538,-0.863868,2.639132,2.559463,0.418405,-0.687494,2.757389,1.082492,-1.553548,-0.270715,2.520047,0.715392,1.751897,0.091667,1.122074,1.836562,0.902718,2.028400,0.694418,0.918927,0.344681,1.900667,2.109940,3.574720,0.612016,2.038527,1.179744,2.504041,1.173482,1.039130,1.528003,1.673750,0.767692,-0.031244,1.354711
std,5.171798,5.597327,5.399806,5.234654,5.255700,4.900540,5.495931,4.929144,5.162224,5.087359,4.916611,4.847879,5.076765,5.122385,5.123290,4.956258,4.398482,4.879306,4.887486,5.100931,4.881075,5.099855,5.110971,5.229222,5.044315,4.856339,4.602144,4.977191,4.792813,4.936876,4.916673,4.456629,5.248554,5.034648,4.597890,4.28162,5.200415,5.011207,4.867559,5.043951,...,4.718655,4.547434,5.143053,5.152378,5.120954,4.701836,5.216555,4.621860,4.662009,4.771693,5.808941,4.529657,5.224221,5.032757,5.427726,4.724249,5.143225,4.932234,5.307042,5.523134,5.338919,5.054666,5.104706,5.118793,5.290301,5.133460,4.782956,4.703183,4.799183,5.259641,5.060445,5.127291,4.611339,5.140307,5.167043,4.852069,5.025076,5.393833,5.176806,5.300409
min,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.95000,-9.950000,-9.950000,-9.950000,-9.950000,...,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000
25%,-2.670000,-4.420000,-4.030000,-5.870000,-3.690000,-1.310000,-5.290000,-4.610000,-4.760000,-2.090000,-1.210000,-1.650000,-6.260000,-2.090000,-6.210000,-7.430000,-4.420000,-4.420000,-3.450000,-5.050000,-0.780000,-2.820000,-3.790000,-6.210000,-3.350000,-1.840000,0.680000,-1.650000,0.340000,-4.170000,-0.630000,0.630000,-5.920000,-2.770000,0.390000,0.97000,-5.870000,-2.140000,-2.140000,-2.520000,...,-0.290000,0.440000,-3.640000,-4.810000,-0.780000,-0.24

In [11]:
df.shape

(24983, 100)

In [12]:
df.head(15)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.42,NaN,NaN,NaN,-7.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,7.91,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,NaN,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,NaN,8.25,NaN,NaN,7.48,7.28,7.28,8.93,NaN,6.17,7.28,NaN,NaN,8.98,7.33,NaN,6.17,9.08,7.33,...,6.46,7.28,NaN,NaN,7.04,7.28,NaN,7.28,8.25,NaN,NaN,NaN,NaN,NaN,NaN,8.93,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.03,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,NaN,NaN,NaN,NaN,2.91,3.93,6.75,6.60,NaN,6.65,-6.12,NaN,7.57,6.21,6.65,NaN,-8.30,7.18,2.82,...,0.00,-3.69,NaN,NaN,7.82,0.24,NaN,7.28,-2.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.63,NaN,NaN,-2.33,NaN,NaN,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,2.48,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,NaN,NaN,NaN,NaN,NaN,NaN,4.13,NaN,NaN,NaN,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
5,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,-1.80,-6.80,-5.73,-5.00,-8.59,0.49,-8.93,-3.69,-2.18,-2.28,-6.12,-3.01,-0.58,-2.38,-7.77,-2.23,-2.28,-5.24,-5.53,-0.68,-6.94,-3.93,-0.34,-6.60,-5.49,-1.55,-0.49,-6.21,-8.06,-5.39,-7.04,...,-6.65,-2.86,-4.61,-9.42,-7.91,-9.81,-9.42,-8.98,-1.94,-6.99,-9.66,-5.19,-5.00,-4.42,-7.28,-6.50,-4.32,-6.94,-9.85,0.73,-6.21,-2.86,-7.62,-6.12,-0.29,-1.41,-3.93,-1.94,-5.68,-4.71,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45
6,NaN,NaN,NaN,NaN,8.59,-9.85,7.72,8.79,NaN,NaN,4.27,7.62,-6.26,2.96,6.07,-3.50,-2.09,6.17,5.15,4.42,5.63,2.43,NaN,NaN,4.13,7.09,7.33,7.18,0.92,NaN,5.15,5.87,NaN,NaN,7.96,3.79,NaN,1.55,NaN,3.11,...,-1.50,5.78,NaN,NaN,8.93,8.50,NaN,2.43,7.48,NaN,NaN,5.97,6.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.33,NaN,NaN,NaN,NaN
7,6.84,3.16,9.17,-6.21,-8.16,-1.70,9.27,1.41,-5.19,-4.42,8.20,-7.86,-6.94,-7.96,0.29,-9.90,-7.09,-7.18,1.02,-0.29,-4.71,-7.43,1.12,-8.25,3.79,0.10,3.45,-3.45,7.57,-4.17,-0.44,9.27,5.83,-3.40,-5.44,1.60,-7.14,3.01,-6.07,5.68,...,-3.20,-6.12,-2.91,-9.22,-2.62,-7.33,-3.25,-4.22,-9.81,-2.09,-9.90,-3.79,0.49,-7.86,-9.85,1.80,-4.17,-2.38,-9.13,-8.88,3.54,-4.56,-5.44,-5.97,-9.90,-0.34,-4.13,-0.10,-0.24,-7.96,7.23,-1.12,-0.10,-5.68,-3.16,-3.35,2.14,-0.05,1.31,0.00
8,-3.79,-3.54,-9.42,-6.89,-8.74,-0.29,-5.29,-8.93,-7.86,-1.60,-2.91,-0.29,-4.85,-0.49,-8.74,-6.99,-8.74,-2.91,-3.35,-0.29,3.98,-1.60,-0.29,1.21,3.79,2.91,4.85,2.28,0.97,-0.29,6.36,0.53,-0.29,4.66,3.35,0.97,-3.11,-1.84,4.42,-4.42,...,1.21,5.53,1.41,-6.50,4.85,4.85,2.62,1.41,6.36,1.41,-4.76,4.17,-2.04,-6.31,-0.29,-0.29,0.78,-0.29,-0.29,-0.29,-0.29,-3.20,-0.29,-3.20,-3.40,-0.29,-0.29,5.73,4.56,-2.23,4.37,-0.29,4.17,-0.29,-0.29,-0.29,-0.29,-0.29,-3.40,-4.9

In [13]:
#To find out the average rating of the lowest/highest
joke_mean_sorted = sorted(df.mean())
joke_mean_max = max(joke_mean_sorted)
joke_mean_min = min(joke_mean_sorted)

print('Max ', joke_mean_max, 'Min ', joke_mean_min)
#print('Full list sorted: ')
#joke_mean_sorted

Max  3.6650848950824937 Min  -3.8338796373689923


In [14]:
df.min().min()

-9.95

In [16]:
#Finds specific value of user and joke
df.iloc[6075,35]

2.62

In [17]:
arr = df.values
arr

array([[-7.82,  8.79, -9.66, ...,   nan,   nan,   nan],
       [ 4.08, -0.29,  6.36, ...,  0.34, -4.32,  1.07],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [ 2.43,  2.67, -3.98, ...,   nan,   nan,   nan]])

In [18]:
def replace(orig, percentage):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """

  new_data = orig.copy()
  rated = np.where(np.isnan(arr))
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [19]:
user_ratings,idx = replace(arr, 0.1)

In [20]:
user_ratings

array([[-7.82,  8.79, -9.66, ...,   nan,   nan,   nan],
       [ 4.08, -0.29,  6.36, ...,  0.34, -4.32,  1.07],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan, 99.  ],
       [  nan, 99.  , 99.  , ...,   nan,   nan, 99.  ],
       [ 2.43,  2.67, -3.98, ...,   nan,   nan,   nan]])

In [21]:
n_latent_factors = 4

latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [23]:
latent_user_preferences

array([[0.18992783, 0.06666254, 0.68671834, 0.98633159],
       [0.8631545 , 0.2518033 , 0.14166336, 0.51266218],
       [0.34579811, 0.8376381 , 0.04407484, 0.7471199 ],
       ...,
       [0.9714835 , 0.0080286 , 0.78895015, 0.63955262],
       [0.44179039, 0.28684629, 0.10643828, 0.22700967],
       [0.50241238, 0.16777109, 0.13717978, 0.42183924]])

In [24]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id, item_id]
                if (rating != 99 and not np.isnan(rating)):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
        mse_history.append(mse)
    return mse_history

In [25]:
sgd(150)

Iteration 0/150:	MSE=25.629424
Iteration 1/150:	MSE=24.972801
Iteration 2/150:	MSE=24.838039
Iteration 3/150:	MSE=24.713616
Iteration 4/150:	MSE=24.588210
Iteration 5/150:	MSE=24.461020


KeyboardInterrupt: ignored

In [26]:
#prediction for validation set
error_list = []
for i,j in zip(idx[0],idx[1]):
  error_list.append(predict_rating(i,j) - arr[i,j])

final_mse = (np.array(error_list)**2).mean()
print(final_mse)

nan


In [27]:
#prediction for nans
np.where(np.isnan(arr))

for i,j in zip(np.where(np.isnan(arr))[0], np.where(np.isnan(arr))[1]):
  arr[i,j] = predict_rating(i,j)

In [28]:
np.where(np.isnan(arr))

(array([], dtype=int64), array([], dtype=int64))

In [29]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[0.95424289, 0.46701967, 0.53438235, ..., 1.08508212, 0.1872029 ,
        1.54233347],
       [0.66638764, 0.29722807, 0.37087682, ..., 1.00544884, 0.29776825,
        1.19651376],
       [1.21840101, 0.55706146, 0.71184026, ..., 1.47532385, 0.46471216,
        2.02533925],
       ...,
       [0.95574251, 0.45726844, 0.46586402, ..., 1.2287122 , 0.24417972,
        1.64869119],
       [0.49537439, 0.22413467, 0.26344588, ..., 0.65683978, 0.20130018,
        0.86199748],
       [0.68250049, 0.31281946, 0.37942142, ..., 0.92301962, 0.25588438,
        1.1792985 ]])

In [30]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [31]:
comparison_data
# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, 0.9542428861823574)","(8.79, 0.46701967174156017)","(-9.66, 0.5343823518582439)","(-8.16, -1.0186742185579118)","(-7.52, 0.715061231092837)","(-8.5, 1.8879110078170516)","(-9.85, -0.3304318241570866)","(4.17, -0.34854723869986365)","(-8.98, -0.3464244174639638)","(-4.76, 1.5122166633715652)","(-8.5, 1.8647150474827972)","(-6.75, 1.6901602293799314)","(-7.18, -1.3156119387470286)","(8.45, 1.5337488601836995)","(-7.18, -1.318079072109481)","(-7.52, -2.6810002677617906)","(-7.43, -0.777902296745207)","(-9.81, -0.43349729256253655)","(-9.85, 0.43476257089792175)","(-9.85, -0.45786541620770205)","(-9.37, 2.223960273884115)","(1.5, 1.0658223922311134)","(-4.37, 0.6182104298599694)","(-9.81, -0.9803640907555841)","(-8.5, 0.8000103047516249)","(1.12, 1.5782907396386523)","(7.82, 3.2592320330318834)","(2.86, 1.8152215186320797)","(9.13, 2.889118756348189)","(-7.43, 0.18542316917908544)","(2.14, 2.276475160691528)","(-4.08, 3.0865084241090135)","(-9.08, -0.972872675371772)","(7.82, 1.1228243314639534)","(5.05, 3.0597476187758463)","(4.95, 3.013773778104292)","(-9.17, -0.9273727871234043)","(-8.4, 1.4673472035085926)","(-8.4, 1.3388329289809033)","(-8.4, 1.354818973574146)",...,"(8.59, 2.6973407068866635)","(3.59, 3.0236964197447858)","(-6.84, 0.8137876305377443)","(-9.03, -0.2869303236805225)","(2.82, 2.4027029200429078)","(-1.36, 2.684635945345901)","(-9.08, -0.39991413058753256)","(8.3, 2.756367736154678)","(5.68, 2.6095675650770427)","(-4.81, 0.7491207086334418)","(nan, -0.25808791316428126)","(99.0, 2.503119543251334)","(nan, 1.1127350690847018)","(99.0, -0.8550542947742859)","(nan, 0.08666335406578696)","(nan, 2.4725266289815697)","(nan, 1.1430924622244143)","(-9.42, 1.806374918800432)","(nan, 0.7219091192078423)","(99.0, 1.3039281982828526)","(nan, 1.839227451069273)","(-7.72, 0.7563493267995781)","(nan, 1.781672530257989)","(nan, 0.9301838122783174)","(nan, 1.0641200791688956)","(nan, 0.3628181698181504)","(nan, 1.8850316211600737)","(nan, 2.2170531602825845)","(nan, 3.7079805313012817)","(nan, 0.9440160233285034)","(2.82, 2.0393393708548975)","(nan, 1.5444349099093602)","(nan, 2.3908060014997186)","(nan, 1.325964127829602)","(nan, 0.8840519705395744)","(99.0, 1.6928567035666444)","(-5.63, 1.6804837902062895)","(nan, 1.0850821167049458)","(nan, 0.1872028959156565)","(nan, 1.5423334674228586)"
1,"(4.08, 0.666387644383855)","(-0.29, 0.29722807285282377)","(6.36, 0.3708768171322821)","(4.37, -1.0788647888644654)","(-2.38, 0.4812593949974862)","(-9.66, 1.3632794615517048)","(-0.73, -0.19234300135026383)","(-5.34, -0.4466093167469462)","(8.88, -0.09462072614863624)","(9.22, 1.1620182507607597)","(6.75, 1.6327148218880465)","(8.64, 1.4042027545679894)","(4.42, -1.351121989645317)","(7.43, 1.284707712641844)","(4.56, -1.3785603456899729)","(-0.97, -2.496269142450545)","(4.66, -0.7556592333432549)","(-0.68, -0.40331285010353685)","(3.3, 0.24038581109938242)","(-1.21, -0.4636646238370748)","(0.87, 1.78744700442197)","(8.64, 0.9298571287795618)","(8.35, 0.2799959190902554)","(9.17, -1.1683042451166898)","(0.05, 0.48453057494003277)","(7.57, 1.2515082634950974)","(4.71, 2.551181485889923)","(0.87, 1.4262796194250653)","(-0.39, 2.496356611633348)","(6.99, -0.08166859203744302)","(6.5, 1.850488282864478)","(-0.92, 2.6595357955683143)","(7.14, -0.7402455755147106)","(9.03, 1.0142506655038015)","(-1.8, 2.5668209461073794)","(0.73, 2.7190348062736076)","(7.09, -0.9609641496696872)","(3.4, 1.3202970769869236)","(-0.87, 1.022369836393503)","(7.91, 0.9255956216157035)",...,"(-6.7, 2.1268353661099777)","(-3.35, 2.483256584629617)","(-9.03, 0.3675198113641369)","(4.47, -0.417703084350297)","(4.08, 2.0927361482108866)","(-3.83, 2.0648731439313392)","(8.74, -0.44836701147434355)","(1.12, 2.2964049214289535)",

In [32]:
#movies dataset

#Matrix for movies and their factors
movie_items_data = pd.read_excel('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/movies_latent_factors.xlsx', sheet_name='Items')
#Matrix for users and their factors
movie_users_data = pd.read_excel('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/movies_latent_factors.xlsx', sheet_name='Users')

In [33]:
movie_users_data.head(5)

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,4768,-0.204024,0.161079,-0.090447,0.138495,-0.162934,0.163894,0.051502,-0.088582,0.126829,0.065967,0.085008,0.355404,0.007108,-0.118663,-0.039125
1,156,-0.189652,-0.178979,-0.091490,-0.000823,-0.032646,0.177209,-0.098123,-0.068283,-0.011575,0.120866,-0.009931,-0.048606,0.045916,0.113671,0.179873
2,5323,-0.115308,-0.090886,-0.053129,0.018472,-0.068081,-0.004828,0.113005,0.102107,0.034758,0.000693,-0.073712,-0.019460,0.108372,0.054471,-0.109552
3,174,-0.227462,-0.272532,-0.017231,0.054324,0.214755,-0.072639,-0.033122,-0.086508,-0.131479,0.180403,0.095890,-0.082396,0.036767,-0.165438,0.050692
4,4529,-0.014616,-0.102218,-0.107935,0.155784,-0.123362,-0.118228,-0.013549,-0.050622,0.058698,-0.159600,-0.142382,-0.132836,-0.039897,0.129063,0.102669


In [34]:
#Selects the whole row of the specified Movie ID
movie_items_data[movie_items_data['Movie ID'] == 4327]

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
86,4327,Charlie's Angels (2000),4.023815,0.882963,1.138274,-0.431058,0.208086,-0.853887,-0.590821,0.416106,0.38739,0.522155,-0.379812,-0.567725,-1.007534,0.794589,-2.247246


In [35]:
#Selects the whole row of the specified User
movie_users_data[movie_users_data['User'] == 4327]

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
21,4327,0.008227,0.073899,-0.323967,0.131203,-0.320862,0.054997,0.060751,0.162314,0.044612,-0.232241,0.094986,-0.10657,-0.051692,-0.09349,0.029534


In [36]:
#Selects the whole row of the specified Movie Factor with the minimum loading score (replace .min with .max for maximum)
movie_items_data[movie_items_data['Factor14'] == movie_items_data['Factor14'].min() ]

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
43,558,Spider-Man 2 (2004),2.235223,-0.788129,0.596722,-0.400082,-1.246623,-1.507203,-0.219911,-0.486254,0.331657,-1.833755,-2.24334,-1.197938,-1.450812,-2.381119,-0.455753


In [37]:
#Calculates the dot product between the User,movie and the factor. Change User/Movie/factor to compare two
np.array(movie_users_data[movie_users_data['User'] == 783].loc[:,'Factor1':]).dot(np.array(movie_items_data[movie_items_data['Movie ID']==597].loc[:,'Factor1':]).reshape(-1,1))

array([[0.05219643]])

In [39]:
def recommend_movie(user, num_recommendations):
  user = np.array(movie_users_data[movie_users_data['User'] == user].loc[:,'Factor1':])
  scores = pd.DataFrame(columns=['ratings','Movie'])
  movie_id = movie_items_data['Movie ID']
  count = 0
  for i in movie_items_data.drop(['Movie ID'],axis=1).values:
    rating = user.dot(i[1:].reshape(-1,1))
    movie_dict = {'ratings':rating,'Movie':i[0],'Movie ID':movie_id[count]}
    scores = scores.append(movie_dict, ignore_index=True)
    count += 1
  return scores.sort_values('ratings', ascending=False).head(num_recommendations)

In [41]:
#Recommends movies based on User and how many recommendations are required
recommend_movie(4327, 3)

,ratings,Movie,Movie ID
47,[[1.386042222743541]],Titanic (1997),597.0
35,[[1.0675675245300227]],Jurassic Park (1993),329.0
2,[[0.9572237810856431]],Forrest Gump (1994),13.0
